In [1]:
import syft as sy
import torch as th
import nest_asyncio
from syft.grid.duet.om_signaling_client import register
from syft.grid.duet.webrtc_duet import Duet
import sys
nest_asyncio.apply()
sy.VERBOSE = False

def begin_duet_logger(my_domain):
    import sys
    import threading
    import time
    from contextlib import contextmanager

    # we need a lock, so that other threads don't snatch control
    # while we have set a temporary parent
    stdout_lock = threading.Lock()

    @contextmanager
    def set_stdout_parent(parent):
        """a context manager for setting a particular parent for sys.stdout

        the parent determines the destination cell of output
        """
        save_parent = sys.stdout.parent_header
        with stdout_lock:
            sys.stdout.parent_header = parent
            try:
                yield
            finally:
                # the flush is important, because that's when the parent_header actually has its effect
                sys.stdout.flush()
                sys.stdout.parent_header = save_parent

    class counterThread(threading.Thread):
        def run(self):
            # record the parent when the thread starts
            thread_parent = sys.stdout.parent_header
            iterator = 0
            while(True):
                time.sleep(0.1)
                # then ensure that the parent is the same as when the thread started
                # every time we print
                with set_stdout_parent(thread_parent):
                    
                    n_objects = len(my_domain.store)
                    n_requests = len(my_domain.requests)
                    n_messages = my_domain.message_counter
                    
                    blink_on = (int(iterator / 5) % 2) == 0
                    
                    if blink_on and n_requests > 0:
                        left_blink = ">"
                        right_blink = "<"
                    else:
                        left_blink = " "
                        right_blink = " "
                        
                    if blink_on:
                        star = "*"
                    else:
                        star = "-"
                    
                    out = "♫♫♫ > DUET LIVE STATUS  "+star+"  Objects: " + str(n_objects) + "  Requests:"+left_blink + str(n_requests) + right_blink+ "  Messages: " + str(n_messages)
                    
                    sys.stdout.write("\r" + out)        
                iterator += 1
                    
                
    counterThread().start()

def launch_duet(logging=True, network_url="http://ec2-18-191-23-46.us-east-2.compute.amazonaws.com:5000"):

    print("🎤  🎸  ♪♪♪ starting duet ♫♫♫  🎻  🎹\n")
    sys.stdout.write(
        "♫♫♫ >\033[93m" + " DISCLAIMER" + "\033[0m"
        ":"
        + "\033[1m"
        + " Duet is an experimental feature currently \n♫♫♫ > "
        + "in alpha. Do not use this to protect real-world data.\n"
        + "\033[0m"
    )

    print("♫♫♫ >")
    print("♫♫♫ > Punching through firewall to signaling client: ")
    print("♫♫♫ > " + str(network_url))
    print("♫♫♫ >")
    sys.stdout.write("♫♫♫ > ...waiting for response... ")

    signaling_client = register(url=network_url)

    print("DONE!")
    
#     print("♫♫♫ >")
#     print("♫♫♫ > Your Duet Id: " + signaling_client.duet_id)    
    
    print("♫♫♫ >")
    print("♫♫♫ > STEP 1: Send the following code to your duet partner!")    
#         print(f"♫♫♫ > Duet Node ID:{domain.id.value}")
    
    print("\n import syft as sy")
    print("\n sy.join_duet('"+signaling_client.duet_id+"')")    

    my_domain = sy.Domain(name="Launcher")
    
    print("\n♫♫♫ > Step 2: The code above will print out a 'Client Id'. Have")
    print("♫♫♫ > your duet partner send it to you and enter it below!")
    print()
    target_id = input("♫♫♫ > Duet Partner's Client Id:")

    duet = Duet(node=my_domain,
                target_id=target_id,
                signaling_client=signaling_client,
                offer=True)
    
#     return duet, my_domain.get_root_client()
    out_duet = my_domain.get_root_client()
    
    if(logging):
        begin_duet_logger(my_domain)
    print()
    return out_duet

In [2]:
duet = launch_duet()

🎤  🎸  ♪♪♪ starting duet ♫♫♫  🎻  🎹

♫♫♫ > DISCLAIMER: Duet is an experimental feature currently 
♫♫♫ > in alpha. Do not use this to protect real-world data.
♫♫♫ >
♫♫♫ > Punching through firewall to signaling client: 
♫♫♫ > http://ec2-18-191-23-46.us-east-2.compute.amazonaws.com:5000
♫♫♫ >
♫♫♫ > ...waiting for response... DONE!
♫♫♫ >
♫♫♫ > STEP 1: Send the following code to your duet partner!

 import syft as sy

 sy.join_duet('a3984657536fe7b9cbe3535ee016e380')

♫♫♫ > Step 2: The code above will print out a 'Client Id'. Have
♫♫♫ > your duet partner send it to you and enter it below!



♫♫♫ > Duet Partner's Client Id: 9d75faca18310d0535db3f29674d2ec3


♫♫♫ > Using a running event loop

♫♫♫ > DUET LIVE STATUS  *  Objects: 1  Requests: 0   Messages: 7

In [3]:
duet.requests.pandas

""


In [4]:
x = th.tensor([1,2,3]).tag("data").send(duet, searchable=True)

Exception in callback AsyncIOEventEmitter._emit_run.<locals>._callback(<Task finishe... a request.')>) at /Users/atrask/opt/anaconda3/envs/syft/lib/python3.7/site-packages/pyee/_asyncio.py:55
handle: <Handle AsyncIOEventEmitter._emit_run.<locals>._callback(<Task finishe... a request.')>) at /Users/atrask/opt/anaconda3/envs/syft/lib/python3.7/site-packages/pyee/_asyncio.py:55>
Traceback (most recent call last):
  File "/Users/atrask/opt/anaconda3/envs/syft/lib/python3.7/site-packages/nest_asyncio.py", line 190, in run
    ctx.run(self._callback, *self._args)
  File "/Users/atrask/opt/anaconda3/envs/syft/lib/python3.7/site-packages/pyee/_asyncio.py", line 62, in _callback
    self.emit('error', exc)
  File "/Users/atrask/opt/anaconda3/envs/syft/lib/python3.7/site-packages/pyee/_base.py", line 111, in emit
    self._emit_handle_potential_error(event, args[0] if args else None)
  File "/Users/atrask/opt/anaconda3/envs/syft/lib/python3.7/site-packages/pyee/_base.py", line 83, in _emit_handl

In [7]:
duet.requests[0].accept()

In [11]:
duet.requests.pandas

""


In [8]:
import sys

In [10]:
duet.store.pandas

,ID,Tags,Description
0,<UID:f2f38646-72e9-4f7c-8efe-3af2b7273257>,[],
1,<UID:041c3df4-3da8-4466-b252-e3a7a5026349>,[],
2,<UID:a501ff1f-2532-4f84-b2c6-030b0898604a>,[data],
3,<UID:727669aa-f0cb-442a-87ad-62bd35cfb302>,[],
4,<UID:416fce23-528d-4fca-9880-b3a49293835c>,[],
5,<UID:6b5e1155-da18-4f0b-9faa-79a099b0c317>,[],
6,<UID:808be15b-66f5-480d-8e5a-2507f7f40e7d>,[],
7,<UID:dbc93b21-b6e4-4e21-9958-aefdf50e011a>,[],
8,<UID:856368ce-52fa-4fe3-83a2-6b9c110cc889>,[],
9,<UID:d07a8a21-a5b2-4cee-b77a-fdb69bb14367>,[],


In [14]:
print("Send/Request remote operations")
x = th.tensor([1,2,3,4,5,6])
p_x = x.send(duet)
for i in range(10):
    p_x = p_x + p_x
print("My Result: ", p_x.get())

Send/Request remote operations
My Result:  tensor([1024, 2048, 3072, 4096, 5120, 6144])


In [3]:
signaling_client2 = register(url="http://ec2-18-222-9-218.us-east-2.compute.amazonaws.com:5000")
my_domain2 = sy.Domain(name="Bob2")
print("Please enter ID of node you're connecting to!")
target_id2 = input()

duet2 = Duet(node=my_domain2,
            target_id=target_id2,
            signaling_client=signaling_client2,
            offer=True)

Duet ID:  ad824e6a7c9c483e138e175ba223bcf7
Please enter ID of node you're connecting to!


 820ad28ebb50f10356cf155cdd1c68cb


> Using a running event loop


In [4]:
print("Send/Request remote operations")
x = th.tensor([1,2,3,4,5,6])
p_x = x.send(duet)
for i in range(10):
    p_x = p_x + p_x
print("My Result: ", p_x.get())

Send/Request remote operations
My Result:  tensor([1024, 2048, 3072, 4096, 5120, 6144])


In [10]:
print("Send/Request remote operations")
x = th.tensor([1,2,3,4,5,6])
p_x = x.send(duet2)
for i in range(1000):
    p_x = p_x + 1
print("My Result: ", p_x.get())

Send/Request remote operations
My Result:  tensor([1001, 1002, 1003, 1004, 1005, 1006])


## Create Our Domain
Now, we need to create our domain which will work as a server, computing remote requests.

## Define Target's Domain/Address
PS: We don't need to create a domain to our target peer (just its address would be enough).

## Start Duet Process
Now, we can start our ICE protocol, if everything goes well, at the end of this process we'll be connected
with the other side and ready for send/receive messages.

**PS: It is important to pay attention to the "offer" parameter,if this parameter is the same for both peers, communication will not be established.**

> Using a running event loop


## Send/Receive Requests/Responses
The great advantage of a full-duplex channel is: You can act as a server and as a client simultaneously.

In [5]:
print("Send/Request remote operations")
x = th.tensor([1,2,3,4,5,6])
p_x = x.send(duet)
for i in range(10):
    p_x = p_x + p_x
print("My Result: ", p_x.get())

Send/Request remote operations
My Result:  tensor([1024, 2048, 3072, 4096, 5120, 6144])


In [ ]:
duet.close()